<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/Strong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta requests pandas numpy --quiet

import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange

# -------------------------- ⚙️ НАСТРОЙКИ --------------------------
TELEGRAM_TOKEN = "8111633898:AAEH_ssxu0KiyTNHbZ2Ig04ilSTCRGCV6b8"
CHAT_ID = 381202205

BASE_URL = "https://www.okx.com/api/v5"
HEADERS = {"User-Agent": "Mozilla/5.0"}

INTERVALS = ["1m", "5m", "15m", "30m"]
VOLATILITY_THRESHOLD = 0.003
PAUSE_BETWEEN_COINS = 1.5
SLEEP_MINUTES = 10

last_signals = {}

# -------------------------- 📨 TELEGRAM --------------------------
def send_telegram(message):
    try:
        url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
        payload = {"chat_id": CHAT_ID, "text": message, "parse_mode": "HTML"}
        requests.post(url, data=payload)
    except Exception as e:
        print("Ошибка Telegram:", e)

# -------------------------- 📊 Получение списка монет --------------------------
def fetch_usdt_futures():
    try:
        r = requests.get(f"{BASE_URL}/public/instruments?instType=SWAP", headers=HEADERS).json()
        return [d["instId"] for d in r.get("data", []) if "USDT" in d["instId"]]
    except Exception as e:
        print("Ошибка загрузки списка:", e)
        return []

# -------------------------- 📈 Анализ одной монеты --------------------------
def analyze_symbol(symbol):
    scores = []
    for tf in INTERVALS:
        try:
            url = f"{BASE_URL}/market/candles?instId={symbol}&bar={tf}&limit=200"
            r = requests.get(url, headers=HEADERS, timeout=10).json()
            data = r.get("data", [])
            if not data:
                continue
            df = pd.DataFrame(data, columns=["ts","o","h","l","c","v","vCcy","vUsd","confirm"])
            df = df.astype({"o":float,"h":float,"l":float,"c":float,"v":float})
            df = df.iloc[::-1].reset_index(drop=True)
            if len(df) < 50:
                continue

            atr = AverageTrueRange(df["h"], df["l"], df["c"], window=14).average_true_range()
            volatility = atr.iloc[-1] / df["c"].iloc[-1]
            if volatility < VOLATILITY_THRESHOLD:
                return None

            sma = SMAIndicator(df["c"], 20).sma_indicator()
            ema = EMAIndicator(df["c"], 50).ema_indicator()
            macd = MACD(df["c"]).macd_diff()
            rsi = RSIIndicator(df["c"]).rsi()

            price = df["c"].iloc[-1]
            score = 0
            if sma.iloc[-1] > ema.iloc[-1]: score += 1
            if macd.iloc[-1] > 0: score += 1
            if rsi.iloc[-1] < 30: score += 1
            if sma.iloc[-1] < ema.iloc[-1]: score -= 1
            if macd.iloc[-1] < 0: score -= 1
            if rsi.iloc[-1] > 70: score -= 1
            scores.append(score)
        except Exception:
            continue

    if not scores:
        return None

    avg_signal = np.mean(scores)
    atr_val = atr.iloc[-1]
    if avg_signal >= 1.5:
        signal = "🟢 СИЛЬНЫЙ ЛОНГ"
        sl = price - 2 * atr_val
        tp = price + 3 * atr_val
    elif 0.5 <= avg_signal < 1.5:
        signal = "🟩 ЛОНГ"
        sl = price - 1.5 * atr_val
        tp = price + 2.5 * atr_val
    elif -1.5 <= avg_signal <= -0.5:
        signal = "🟥 ШОРТ"
        sl = price + 1.5 * atr_val
        tp = price - 2.5 * atr_val
    elif avg_signal < -1.5:
        signal = "🔴 СИЛЬНЫЙ ШОРТ"
        sl = price + 2 * atr_val
        tp = price - 3 * atr_val
    else:
        return None

    return {"symbol": symbol, "signal": signal, "price": round(price,4), "sl": round(sl,4), "tp": round(tp,4)}

# -------------------------- 🔁 Главный цикл --------------------------
def run_bot():
    global last_signals
    print("🚀 Бот запущен. Проверка рынка каждые 10 минут.")
    send_telegram("🤖 Бот запущен. Проверка рынка каждые 10 минут.")

    symbols = fetch_usdt_futures()
    if not symbols:
        print("❌ Не удалось загрузить список фьючерсов.")
        return

    while True:
        start_time = datetime.utcnow().strftime("%H:%M:%S")
        print(f"\n🕓 {start_time} — начало анализа {len(symbols)} инструментов")

        new_signals = []
        for i, sym in enumerate(symbols[:80]):  # ограничим до 80 монет
            print(f"[{i+1}/{len(symbols)}] {sym}", end="\r")
            result = analyze_symbol(sym)
            if not result:
                continue

            key = f"{result['symbol']}_{result['signal']}"
            if key != last_signals.get(result["symbol"]):
                new_signals.append(result)
                last_signals[result["symbol"]] = key

        if new_signals:
            msg = f"📊 <b>{len(new_signals)} новых сигналов</b>:\n\n"
            for s in new_signals:
                msg += f"{s['symbol']}: {s['signal']}\nЦена: {s['price']}\nSL: {s['sl']}\nTP: {s['tp']}\n\n"
            send_telegram(msg)
            print(msg)
        else:
            print("📭 Новых сигналов нет.")

        print(f"⏳ Ожидание {SLEEP_MINUTES} мин...\n")
        for sec in range(SLEEP_MINUTES * 60):
            time.sleep(1)

# -------------------------- ▶️ Запуск --------------------------
run_bot()

  Preparing metadata (setup.py) ... done
🚀 Бот запущен. Проверка рынка каждые 10 минут.


/tmp/ipython-input-3298984960.py:120: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = datetime.utcnow().strftime("%H:%M:%S")



🕓 15:57:26 — начало анализа 237 инструментов
